### [Meanshift and Camshift](https://docs.opencv.org/3.4/db/df8/tutorial_py_meanshift.html)
We have already seen an example of color-based tracking. It is simpler. This time, we see significantly better algorithms like "Meanshift", and its upgraded version, "Camshift" to find and track them.

#### Goal
In this chapter,

* We will learn about Meanshift and Camshift algorithms to find and track objects in videos.

#### Meanshift
어지럼 연습 뒤에 있는 직감은 간단 합니다. 너는 일련의 점들을 가지고 있다고 생각해. (히스토그램 역 투영과 같은 픽셀 분포 일 수 있습니다.) 작은 창(원이 될 수도 있음)이 표시되고 창을 최대 픽셀 밀도 (또는 최대 점 수) 영역으로 이동해야합니다. 아래에 제시된 간단한 이미지로 설명됩니다.

![](meanshift_basics.jpg)

초기 창은 이름이 "C1"인 파란색 원으로 표시됩니다. 원래의 중심은 "C1_o"라는 파란색 직사각형으로 표시됩니다. 그러나 그 창 안의 점들의 중심을 발견하면, 당신은 점의 "C1_r"(작은 파란색 원으로 표시됨)을 얻을 것입니다. 이것은 사실의 창 중심입니다. 확실히 그들은 일치하지 않습니다. 따라서 새 창의 원이 이전 중심과 일치하도록 창을 이동하십시오. 다시 새로운 중심을 찾으십시오. 아마, 그것은 일치하지 않을 것입니다. 그래서 다시 움직여서 창의 중심과 중심점이 같은 위치에 오도록 (또는 원하는 작은 오차로) 반복을 반복하십시오. 그래서 마침내 얻을 수있는 것은 픽셀 분포가 최대 인 윈도우입니다. "C2"라는 녹색 원으로 표시됩니다. 이미지에서 볼 수 있듯이 최대 점수가 있습니다.

![](meanshift_face.gif)
그래서 우리는 일반적으로 히스토그램을 다시 투영 한 이미지와 초기 타겟 위치를 전달합니다. 물체가 움직일 때, 분명히 움직임은 히스토그램 백 투영 된 이미지에 반영됩니다. 결과적으로, meanshift 알고리즘은 우리 창을 최대 밀도로 새 위치로 이동시킵니다.

##### Meanshift in OpenCV
OpenCV에서 meanshift를 사용하려면, 먼저 대상을 설정하고 히스토그램을 찾아 각 프레임에서 대상을 되 찾을 수 있도록해야합니다. 우리는 또한 윈도우의 초기 위치를 제공해야합니다. 히스토그램의 경우 색조 만 고려됩니다. 또한 조명 부족으로 인한 잘못된 값을 피하기 위해 [cv.inRange()](https://docs.opencv.org/3.4/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981) 함수를 사용하여 낮은 조명 값을 버립니다 .

In [4]:
import numpy as np
import cv2 as cv

cap = cv.VideoCapture('slow.flv')

# take first frame of the video
ret, frame = cap.read()
temp_frame = frame.copy()

# setup initial location of window
r, h, c, w = 195, 30, 290, 120  # simply hardcoded the values
track_window = (c, r, w, h)

# set up the ROI for tracking
roi      = frame[r:r + h, c:c + w]
hsv_roi  = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask     = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv.calcHist([hsv_roi], [0], mask, [180], [0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
    ret, frame = cap.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0,180], 1)
        
        # apply meanshift to get the new location
        ret, track_window = cv.meanShift(dst, track_window, term_crit)
        
        # Draw it on image
        x, y, w, h = track_window
        img2 = cv.rectangle(frame, (x, y), (x + w, y + h), 255, 2)
        cv.imshow('img2', img2)
        k = cv.waitKey(60) & 0xff
        if k == 27:
            break
        else:
            cv.imwrite(chr(k)+".jpg", img2)
    else:
        break
cv.destroyAllWindows()
cap.release()
cv.waitKey(1)

-1

ROI크기 재설정
내가 사용한 비디오의 세 프레임이 아래에 나와 있습니다.
![](meanshift_result.jpg)

#### Camshift
마지막 결과를주의 깊게 보았습니까? 문제가 있습니다. 자동차가 멀리 떨어져 있고 카메라와 매우 가까운 거리에있을 때 우리 창은 항상 같은 크기입니다. 그건 좋지 않다. 우리는 창 크기를 대상의 크기와 회전으로 조정해야합니다. 다시 한번,이 솔루션은 "OpenCV Labs"에서 왔으며 1998 년 Gary Bradsky가 "Perceptual User Interface에서 사용하기위한 컴퓨터 비전 얼굴 추적"에서 CAMshift (Continuously Adaptive Meanshift)라고 불렀습니다.

그것은 우선적으로 우선 적용됩니다. 일단 수단 쉬프트가 수렴하면, 윈도우의 크기를 \\(s = 2 \times \sqrt{\frac{M_{00}}{256}}\\)으로 갱신합니다. 또한 타원에 가장 잘 맞는 방향을 계산합니다. 다시 말하지만 그것은 새롭게 스케일 된 탐색 창과 이전 창 위치를 갖는 수단 시프트를 적용합니다. 필요한 정확도가 충족 될 때까지 프로세스가 계속됩니다.

![](camshift_face.git)

##### Camshift in OpenCV
그것은 meanhift와 거의 같지만 회전 된 사각형 (즉, 우리의 결과 임)과 상자 매개 변수 (다음 반복에서 검색 창으로 전달되는 데 사용됨)를 반환합니다. 아래 코드를 참조하십시오.

In [2]:
import cv2 as cv
cap = cv.VideoCapture('slow.flv')

# take first frame of the video
ret, frame = cap.read()

# setup initial location of window 
r, h, c, w = 195, 30, 290, 120
#r, h, c, w = 250, 90, 400, 125  # simply hardcoded the values
track_window = (c, r, w, h)

# set up the ROI for tracking
roi      = frame[r:r + h, c:c + w]
hsv_roi  = cv.cvtColor(roi, cv.COLOR_BGR2HSV)
mask     = cv.inRange(hsv_roi, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
roi_hist = cv.calcHist([hsv_roi], [0],mask ,[180] ,[0, 180])
cv.normalize(roi_hist, roi_hist, 0, 255, cv.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
while(1):
    ret ,frame = cap.read()
    if ret == True:
        hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
        dst = cv.calcBackProject([hsv], [0], roi_hist, [0,180], 1)
        
        # apply meanshift to get the new location
        ret, track_window = cv.CamShift(dst, track_window, term_crit)
        
        # Draw it on image
        pts  = cv.boxPoints(ret)
        pts  = np.int0(pts)
        img2 = cv.polylines(frame, [pts], True, 255, 2)
        cv.imshow('img2',img2)
        k = cv.waitKey(60) & 0xff
        if k == 27:
            break
        else:
            cv.imwrite(chr(k)+".jpg",img2)
    else:
        break
cv.destroyAllWindows()
cv.waitKey(1)
cap.release()

Three frames of the result is shown below:

![](camshift_result.jpg)

#### Additional Resources
French Wikipedia page on Camshift. (The two animations are taken from here)
Bradski, G.R., "Real time face and object tracking as a component of a perceptual user interface," Applications of Computer Vision, 1998. WACV '98. Proceedings., Fourth IEEE Workshop on , vol., no., pp.214,219, 19-21 Oct 1998
#### Exercises
OpenCV comes with a Python sample on interactive demo of camshift. Use it, hack it, understand it.

In [1]:
# %load ../python/camshift.py
#!/usr/bin/env python

'''
Camshift tracker
================

This is a demo that shows mean-shift based tracking
You select a color objects such as your face and it tracks it.
This reads from video camera (0 by default, or the camera number the user enters)

http://www.robinhewitt.com/research/track/camshift.html

Usage:
------
    camshift.py [<video source>]

    To initialize tracking, select the object with mouse

Keys:
-----
    ESC   - exit
    b     - toggle back-projected probability visualization
'''

# Python 2/3 compatibility
from __future__ import print_function
import sys
sys.path.insert(0, '../python')
PY3 = sys.version_info[0] == 3

if PY3:
    xrange = range

import numpy as np
import cv2 as cv

# local module
import video
from video import presets


class App(object):
    def __init__(self, video_src):
        self.cam = video.create_capture(video_src, presets['cube'])
        _ret, self.frame = self.cam.read()
        cv.namedWindow('camshift')
        cv.setMouseCallback('camshift', self.onmouse)

        self.selection = None
        self.drag_start = None
        self.show_backproj = False
        self.track_window = None

    def onmouse(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            self.drag_start = (x, y)
            self.track_window = None
        if self.drag_start:
            xmin = min(x, self.drag_start[0])
            ymin = min(y, self.drag_start[1])
            xmax = max(x, self.drag_start[0])
            ymax = max(y, self.drag_start[1])
            self.selection = (xmin, ymin, xmax, ymax)
        if event == cv.EVENT_LBUTTONUP:
            self.drag_start = None
            self.track_window = (xmin, ymin, xmax - xmin, ymax - ymin)

    def show_hist(self):
        bin_count = self.hist.shape[0]
        bin_w = 24
        img = np.zeros((256, bin_count*bin_w, 3), np.uint8)
        for i in xrange(bin_count):
            h = int(self.hist[i])
            cv.rectangle(img, (i*bin_w+2, 255), ((i+1)*bin_w-2, 255-h), (int(180.0*i/bin_count), 255, 255), -1)
        img = cv.cvtColor(img, cv.COLOR_HSV2BGR)
        cv.imshow('hist', img)

    def run(self):
        while True:
            _ret, self.frame = self.cam.read()
            vis              = self.frame.copy()
            hsv              = cv.cvtColor(self.frame, cv.COLOR_BGR2HSV)
            mask             = cv.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))

            if self.selection:
                x0, y0, x1, y1 = self.selection
                hsv_roi        = hsv[y0:y1, x0:x1]
                mask_roi       = mask[y0:y1, x0:x1]
                hist           = cv.calcHist( [hsv_roi], [0], mask_roi, [16], [0, 180] )
                cv.normalize(hist, hist, 0, 255, cv.NORM_MINMAX)
                self.hist = hist.reshape(-1)
                self.show_hist()

                vis_roi = vis[y0:y1, x0:x1]
                cv.bitwise_not(vis_roi, vis_roi)
                vis[mask == 0] = 0

            if self.track_window and self.track_window[2] > 0 and self.track_window[3] > 0:
                self.selection = None
                prob = cv.calcBackProject([hsv], [0], self.hist, [0, 180], 1)
                prob &= mask
                term_crit = ( cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 1 )
                track_box, self.track_window = cv.CamShift(prob, self.track_window, term_crit)

                if self.show_backproj:
                    vis[:] = prob[...,np.newaxis]
                try:
                    cv.ellipse(vis, track_box, (0, 0, 255), 2)
                except:
                    print(track_box)

            cv.imshow('camshift', vis)

            ch = cv.waitKey(100)
            if ch == 27:
                break
            if ch == ord('b'):
                self.show_backproj = not self.show_backproj
        cv.destroyAllWindows()


if __name__ == '__main__':
    import sys
    try:
        video_src = '../python/slow.flv'
    except:
        video_src = 0
    print(__doc__)
    App(video_src).run()



Camshift tracker

This is a demo that shows mean-shift based tracking
You select a color objects such as your face and it tracks it.
This reads from video camera (0 by default, or the camera number the user enters)

http://www.robinhewitt.com/research/track/camshift.html

Usage:
------
    camshift.py [<video source>]

    To initialize tracking, select the object with mouse

Keys:
-----
    ESC   - exit
    b     - toggle back-projected probability visualization

